TRABAJO INDIVIDUAL HENRY N°2
Alumno: Urteaga, Facundo Nahuel
1_EDA : Análisis exploratorio de datos

In [1]:
# Carga de librerías

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Obtener directorios de datos

directorio_actual = os.getcwd()
ruta_hom_hech = os.path.join(directorio_actual, "Datos", "homicidios_hechos.csv")
ruta_hom_vic = os.path.join(directorio_actual, "Datos", "homicidios_victimas.csv")


In [3]:
# Lectura de los archivos .csv

hom_hech_df = pd.read_csv(ruta_hom_hech)
hom_vic_df = pd.read_csv(ruta_hom_vic)

In [ ]:
# Exploro comportamiento de variables y presencia de outliers

# df_hom_hech["N_VICTIMAS"].value_counts() # OK
# df_hom_hech["AAAA"].value_counts() # OK
# df_hom_hech["MM"].value_counts() # OK
# df_hom_hech["DD"].max() # OK
# df_hom_hech["HORA"].max() # Se encuentra que existe un "SD" referente a sin datos.
# df_hom_hech["HH"].min() # Se encuentra que existe un "SD" referente a sin datos.
# df_hom_hech["TIPO_DE_CALLE"].value_counts() # OK
# df_hom_hech["COMUNA"].value_counts() # Hay dos con comuna 0
# df_hom_hech["PARTICIPANTES"].value_counts() # Hay muchos SD por tratar. El resto OK
# df_hom_hech["VICTIMA"].value_counts() # Hay uno por analizar que dice PEATON_MOTO
# df_hom_hech["ACUSADO"].value_counts() # OK

#### 1. Limpieza (?) de hom_hech_df

In [4]:
# Explorar filas y columnas

print('Cantidad de Filas y columnas:',hom_hech_df.shape)
print('Nombre columnas:',hom_hech_df.columns)

Cantidad de Filas y columnas: (696, 21)
Nombre columnas: Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')


In [5]:
# Exploro datos faltantes

hom_hech_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     696 non-null    object 
 1   N_VICTIMAS             696 non-null    int64  
 2   FECHA                  696 non-null    object 
 3   AAAA                   696 non-null    int64  
 4   MM                     696 non-null    int64  
 5   DD                     696 non-null    int64  
 6   HORA                   696 non-null    object 
 7   HH                     696 non-null    object 
 8   LUGAR_DEL_HECHO        696 non-null    object 
 9   TIPO_DE_CALLE          696 non-null    object 
 10  Calle                  695 non-null    object 
 11  Altura                 129 non-null    float64
 12  Cruce                  525 non-null    object 
 13  Dirección Normalizada  688 non-null    object 
 14  COMUNA                 696 non-null    int64  
 15  XY (CA

In [6]:
# Mostrar el número de valores faltantes por columna
print(hom_hech_df.isnull().sum())

# Conclusiones del análisis:

# Hay una calle sin datos debido a que lugar del hecho tiene "sd"
# En general altura y cruce estan intercalados (si no hay altura, quiere decir que fue en un cruce)
# En general donde falta altura o cruce es en las AUTOPISTAS

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64


In [7]:
# Verificar si hay duplicados en el DataFrame
duplicados = hom_hech_df.duplicated()

# Contar el número de registros duplicados
print(f"Número de registros duplicados: {duplicados.sum()}")


Número de registros duplicados: 0


In [8]:
# Convertir las columnas FECHA y HORA a formato datetime especificando el formato correcto

hom_hech_df['FECHA'] = pd.to_datetime(hom_hech_df['FECHA'], format='%m/%d/%Y', errors='coerce')

print(hom_hech_df['FECHA'].isna().sum())

0


In [9]:
print(hom_hech_df.loc[4, 'FECHA'].day)

21


In [10]:
# Convertir la columna HORA a formato de tiempo
hom_hech_df['HORA'] = pd.to_datetime(hom_hech_df['HORA'], format='%H:%M:%S', errors='coerce').dt.time

print(hom_hech_df['HORA'].isna().sum())

1


In [11]:
print(hom_hech_df.loc[4, 'HORA'].hour)

5


In [12]:
# Filtrar las filas donde 'Altura' y 'Calle' están vacías
filas_vacias = hom_hech_df[hom_hech_df['Altura'].isna() & hom_hech_df['Cruce'].isna()]
print(filas_vacias)

            ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA  HH  \
35   2016-0049           1 2016-04-17  2016   4  17  00:00:00   0   
38   2016-0052           1 2016-04-20  2016   4  20  20:00:00  20   
64   2016-0087           1 2016-07-02  2016   7   3  00:10:00   0   
71   2016-0096           1 2016-07-25  2016   7  25  07:00:00   7   
81   2016-0107           1 2016-08-20  2016   8  20  08:22:00   8   
91   2016-0118           1 2016-09-04  2016   9   4  03:30:00   3   
100  2016-0130           1 2016-10-04  2016  10   4  12:30:00  12   
106  2016-0136           1 2016-10-25  2016  10  25  00:00:00   0   
119  2016-0151           1 2016-11-18  2016  11  18  20:35:00  20   
127  2016-0160           1 2016-12-06  2016  12   6  05:30:00   5   
139  2016-0174           1 2016-12-27  2016  12  27  00:00:00   0   
148  2017-0009           1 2017-01-16  2017   1  16  13:56:00  13   
155  2017-0016           1 2017-02-03  2017   2   3  05:12:21   5   
176  2017-0042           1 2017-04

In [13]:
# Mostrar las filas filtradas
print(filas_vacias["LUGAR_DEL_HECHO"].tail())

643                    AU FRONDIZI Y SUAREZ AV.
655    AU ILLIA KM 3.2, ALTURA CABINAS DE PEAJE
665          AU FRONDIZI KM 2,9 (ALTURA BRASIL)
679                 AU FRONDIZI SUBIDA BRANDSEN
680            AU 25 DE MAYO KM7 (ALTURA PASCO)
Name: LUGAR_DEL_HECHO, dtype: object


In [107]:
# Filtrar filas donde 'LUGAR_DEL_HECHO' contiene la palabra 'AUTOPISTA'
#filas_con_autopista = filas_vacias[filas_vacias['LUGAR_DEL_HECHO'].str.contains('AV.', case=False, na=False)]

# Contar el número de filas que cumplen la condición
#numero_filas_con_autopista = filas_con_autopista.shape[0]

#print(f'Número de filas donde "Altura" y "Calle" están vacías y "LUGAR_DEL_HECHO" contiene "AUTOPISTA": {numero_filas_con_autopista}')

In [108]:
# Mediante una búsqueda y análisis exhaustivo, encontramos que aquellas filas que no tienen el dato de "altura" o "cruce" mayoritariamente corresponden a accidentes en autopistas
# Debido a esto, generamos una nueva columna "TIPO_DE_ZONA" que indica si el accidente fue en la cuadra, una esquina, o una autopista.

In [14]:
# Creación de una función para evaluar el tipo de zona del accidente y crear nueva columna

def determinar_tipo_de_zona(row):
    if 'AUTOPISTA' in row['TIPO_DE_CALLE'] or 'GRAL PAZ' in row['TIPO_DE_CALLE']:
        return 'AUTOPISTA'
    elif pd.notna(row['Altura']):
        return 'CUADRA'
    elif pd.notna(row['Cruce']):
        return 'ESQUINA'
    else:
        return None  # O algún valor por defecto si todas las condiciones son falsas


In [15]:
# Crear la nueva columna TIPO_DE_ZONA aplicando la función a cada fila
hom_hech_df['TIPO_DE_ZONA'] = hom_hech_df.apply(determinar_tipo_de_zona, axis=1)

# Verificar las primeras filas para asegurarse de que la columna se creó correctamente
print(hom_hech_df[['Altura', 'Cruce', 'TIPO_DE_CALLE', 'TIPO_DE_ZONA']].head())

   Altura                                Cruce TIPO_DE_CALLE TIPO_DE_ZONA
0     NaN  FERNANDEZ DE LA CRUZ, F., GRAL. AV.       AVENIDA      ESQUINA
1     NaN                  DE LOS CORRALES AV.      GRAL PAZ    AUTOPISTA
2  2034.0                                  NaN       AVENIDA       CUADRA
3     NaN             VILLEGAS, CONRADO, GRAL.       AVENIDA      ESQUINA
4     NaN              SAENZ PE?A, LUIS, PRES.       AVENIDA      ESQUINA


In [16]:
hom_hech_df['TIPO_DE_ZONA'].value_counts()

TIPO_DE_ZONA
ESQUINA      442
AUTOPISTA    131
CUADRA       121
Name: count, dtype: int64

In [17]:
# Completar los valores vacíos en "Dirección Normalizada" con los valores de "LUGAR_DEL_HECHO"
hom_hech_df['Dirección Normalizada'] = hom_hech_df['Dirección Normalizada'].fillna(hom_hech_df['LUGAR_DEL_HECHO'])

In [18]:
# Eliminar las filas donde el valor de la columna "COMUNA" sea igual a 0
hom_hech_df = hom_hech_df.drop(hom_hech_df[hom_hech_df['COMUNA'] == 0].index)

# Verificar que las filas se han eliminado correctamente
print(hom_hech_df['COMUNA'].unique())  # Deberías obtener valores diferentes de 0

[ 8  9  1 11 15  4  7 12  3 13 14 10  6  2  5]


In [19]:
hom_hech_df.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO', 'TIPO_DE_ZONA'],
      dtype='object')

In [20]:
hom_hech_df = hom_hech_df[['ID', 'N_VICTIMAS', 'FECHA', 'HORA', 'TIPO_DE_CALLE','Dirección Normalizada',
                           'TIPO_DE_ZONA', 'COMUNA','PARTICIPANTES', 'VICTIMA', 'ACUSADO', ]]

In [21]:
display(hom_hech_df)

,ID,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,Dirección Normalizada,TIPO_DE_ZONA,COMUNA,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,04:00:00,AVENIDA,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",ESQUINA,8,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,GRAL PAZ,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",AUTOPISTA,9,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,07:00:00,AVENIDA,ENTRE RIOS AV. 2034,CUADRA,1,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,00:00:00,AVENIDA,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",ESQUINA,8,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,05:20:00,AVENIDA,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",ESQUINA,1,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,17:10:00,AVENIDA,RIESTRA AV. y MOM,ESQUINA,7,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,01:10:00,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",AUTOPISTA,9,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,00:43:00,AVENIDA,GAONA AV. y TERRADA,ESQUINA,11,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,10:30:00,AVENIDA,"PERON, EVA AV. 4071",CUADRA,9,AUTO-CARGAS,AUTO,CARGAS


In [24]:
hom_hech_df["N_VICTIMAS"].sum()

715

#### 1. Limpieza (?) de hom_vic_df

In [22]:
# Explorar filas y columnas

print('Cantidad de Filas y columnas:',hom_vic_df.shape)
print('Nombre columnas:',hom_vic_df.columns)

Cantidad de Filas y columnas: (717, 10)
Nombre columnas: Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')


In [25]:
# Exploro datos faltantes

hom_vic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_hecho             717 non-null    object
 1   FECHA                717 non-null    object
 2   AAAA                 717 non-null    int64 
 3   MM                   717 non-null    int64 
 4   DD                   717 non-null    int64 
 5   ROL                  717 non-null    object
 6   VICTIMA              717 non-null    object
 7   SEXO                 717 non-null    object
 8   EDAD                 717 non-null    object
 9   FECHA_FALLECIMIENTO  717 non-null    object
dtypes: int64(3), object(7)
memory usage: 56.1+ KB


In [26]:
# Verificar si hay duplicados en el DataFrame
duplicados = hom_vic_df.duplicated()

# Contar el número de registros duplicados
print(f"Número de registros duplicados: {duplicados.sum()}")


Número de registros duplicados: 0
